In [ ]:
# !pip install sklearn --upgrade

In [25]:
import pandas as pd
from datetime import datetime as dt
import numpy as np
import requests
import config
import json
import time
import datetime
import matplotlib.pyplot as plt
from scipy.stats import linregress
import tabletext
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import sys

from pprint import pprint

In [38]:

df = pd.read_csv('data_files/WBNAfullplayerstatslist.csv')
# covid_playoff = pd.read_csv('data_files/player_stats_2019_pst.csv')

In [ ]:
covid_playoff['player_name'] = covid_playoff['First_Name'].str.cat(covid_playoff['Last_Name'], sep =" ")
# covid_playoff = covid_playoff[['player_name', 'Points','Position','game_date']]
covid_playoff

In [39]:
df['player_name'] = df['First_Name'].str.cat(df['Last_Name'], sep =" ")
# covid_playoff = covid_playoff[['player_name', 'Points','Position','game_date']]
df


,Unnamed: 0,First_Name,Last_Name,player_name,player_id,Points,Free_Throw_Percent,Two_Pt_Percent,Three_Pt_Percent,Assists,...,Offensive_Rebounds,Steals,Personal_Fouls,Flagrant_Fouls,Tech_Fouls,Turnovers,Home_Away,win,Min_played,total_sec_played
0,0,Camille,Little,Camille Little,9951fcfb-8d31-44e5-ba85-c41ed724763d,9,0.0,0.375,20.0,1,...,2,0,2,0,0,2,1,0,29:00:00,1740.0
1,1,Jasmine,Thomas,Jasmine Thomas,0ac8d024-ec11-4809-83f4-6277c1252617,6,100.0,0.250,0.0,4,...,1,0,3,0,0,0,1,0,30:00:00,1800.0
2,2,Kayla,Pedersen,Kayla Pedersen,adfc445c-06c0-444a-8856-32cf13a89ea2,0,0.0,0.000,0.0,0,...,1,0,0,0,0,0,1,0,10:00,600.0
3,4,Elizabeth,Williams,Elizabeth Williams,c8672dba-0deb-4cf1-8304-82365865edaa,5,75.0,0.200,0.0,1,...,2,0,0,0,0,0,1,0,10:00,600.0
4,5,Kelsey,Bone,Kelsey Bone,93a6bb0d-9500-436e-89b4-33bc8ca6909c,14,0.0,0.583,0.0,2,...,1,2,5,0,0,3,1,0,29:00:00,1740.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19235,23685,JiSu,Park,JiSu Park,9a25546d-5f1e-4952-95b7-3e66a76deb92,2,0.0,50.000,0.0,0,...,0,0,2,0,0,0,0,1,4:13,253.0
19236,23686,Liz,Cambage,Liz Cambage,ceee01db-30cb-404f-860c-7acddd358a19,21,60.0,64.300,0.0,2,...,4,1,3,0,0,2,0,1,24:04:00,1444.0
19237,23687,Jackie,Young,Jackie Young,4fa92d21-9f96-4bab-a9f8-f530a69287ee,9,0.0,50.000,100.0,3,...,0,0,2,0,0,1,0,1,16:10,970.0
19238,23688,A'ja,Wilson,A'ja Wilson,6340b055-3c1b-427e-88d6-6f9cdd1fc925,20,100.0,100.000,0.0,1,...,0,0,5,0,0,3,0,1,26:41:00,1601.0


In [ ]:
covid_dropped = covid_playoff[covid_playoff['Min_played'] != "00:00"]
# df_dropped = df_dropped[df_dropped['Crowd'] != 'Covid']
# df_dropped = df_dropped[df_dropped['Crowd'] != '0']
covid_dropped= covid_dropped[['First_Name', 'Last_Name', 'player_name','player_id', "Points", "Free_Throw_Percent",
                  "Two_Pt_Percent",
                  "Three_Pt_Percent", "Assists",
                  "Rebounds", "Offensive_Rebounds",
                  "Steals", "Personal_Fouls",
                  "Flagrant_Fouls", "Tech_Fouls",
                  "Turnovers",
                  "Home_Away", "win", "Min_played"
                  ]].reset_index(drop = True)
covid_dropped.sort_values(by=["Min_played"], ascending = False)

In [40]:
#dropping Crowd from df_dropped to check status weight v win & try to tweat training % up from 71
# df_dropped = df[df['Min_played'] != "00:00"]
# df_dropped = df_dropped[df_dropped['Crowd'] != 'Covid']
# df_dropped = df_dropped[df_dropped['Crowd'] != '0']
df= df[['First_Name', 'Last_Name', 'player_name','player_id', "Points", "Free_Throw_Percent",
                  "Two_Pt_Percent",
                  "Three_Pt_Percent", "Assists",
                  "Rebounds", "Offensive_Rebounds",
                  "Steals", "Personal_Fouls",
                  "Flagrant_Fouls", "Tech_Fouls",
                  "Turnovers",
                  "Home_Away", "win", "Min_played", "Crowd"
                  ]].reset_index(drop = True)
df

KeyError: "['Crowd'] not in index"

In [5]:
df["Min_played"] = df["Min_played"].astype(str)

In [12]:
# time = df['Min_played'].str.split(':', n=2, expand=True)
cols = time.columns
time[cols]= time[cols].apply(pd.to_numeric, errors='coerce')
time.dtypes
time['total_sec_played'] = time[0]*60 + time[1]
time = time['total_sec_played']

df = df.join(time)



In [13]:
time

0        1740.0
1        1800.0
2         600.0
3           NaN
4         600.0
          ...  
23685     253.0
23686    1444.0
23687     970.0
23688    1601.0
23689    1850.0
Name: total_sec_played, Length: 23690, dtype: float64

In [19]:
# df.to_csv('data_files/WBNAfullplayerstatslist.csv')
df.dtypes
df = df[df['total_sec_played'] != 0]
df = df.dropna(subset=['total_sec_played'])
df.to_csv('data_files/WBNAfullplayerstatslist.csv')

df

,First_Name,Last_Name,player_name,player_id,Points,Free_Throw_Percent,Two_Pt_Percent,Three_Pt_Percent,Assists,Rebounds,Offensive_Rebounds,Steals,Personal_Fouls,Flagrant_Fouls,Tech_Fouls,Turnovers,Home_Away,win,Min_played,total_sec_played
0,Camille,Little,Camille Little,9951fcfb-8d31-44e5-ba85-c41ed724763d,9,0.0,0.375,20.0,1,3,2,0,2,0,0,2,1,0,29:00:00,1740.0
1,Jasmine,Thomas,Jasmine Thomas,0ac8d024-ec11-4809-83f4-6277c1252617,6,100.0,0.250,0.0,4,5,1,0,3,0,0,0,1,0,30:00:00,1800.0
2,Kayla,Pedersen,Kayla Pedersen,adfc445c-06c0-444a-8856-32cf13a89ea2,0,0.0,0.000,0.0,0,3,1,0,0,0,0,0,1,0,10:00,600.0
4,Elizabeth,Williams,Elizabeth Williams,c8672dba-0deb-4cf1-8304-82365865edaa,5,75.0,0.200,0.0,1,5,2,0,0,0,0,0,1,0,10:00,600.0
5,Kelsey,Bone,Kelsey Bone,93a6bb0d-9500-436e-89b4-33bc8ca6909c,14,0.0,0.583,0.0,2,6,1,2,5,0,0,3,1,0,29:00:00,1740.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23685,JiSu,Park,JiSu Park,9a25546d-5f1e-4952-95b7-3e66a76deb92,2,0.0,50.000,0.0,0,0,0,0,2,0,0,0,0,1,4:13,253.0
23686,Liz,Cambage,Liz Cambage,ceee01db-30cb-404f-860c-7acddd358a19,21,60.0,64.300,0.0,2,9,4,1,3,0,0,2,0,1,24:04:00,1444.0
23687,Jackie,Young,Jackie Young,4fa92d21-9f96-4bab-a9f8-f530a69287ee,9,0.0,50.000,100.0,3,3,0,0,2,0,0,1,0,1,16:10,970.0
23688,A'ja,Wilson,A'ja Wilson,6340b055-3c1b-427e-88d6-6f9cdd1fc925,20,100.0,100.000,0.0,1,0,0,0,5,0,0,3,0,1,26:41:00,1601.0


In [ ]:
covid_dropped["Min_played"] = covid_dropped["Min_played"].astype(str)
time_covid = covid_dropped['Min_played'].str.split(':', n=1, expand=True)
cols_covid = time_covid.columns
time_covid[cols_covid]= time_covid[cols_covid].apply(pd.to_numeric, errors='coerce')
time_covid.dtypes
time_covid['total_sec_played'] = time_covid[0]*60 + time_covid[1]
time_covid = time_covid['total_sec_played']

covid_dropped = covid_dropped.join(time_covid)



In [ ]:
covid_dropped.to_csv('data_files/covid_dropped.csv')
covid_dropped

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler


In [ ]:
lebron_df = df_dropped['player_id']=='0afbe608-940a-4d5d-a1f7-468718c67d91'
lebron_covid = covid_dropped['player_id']=='0afbe608-940a-4d5d-a1f7-468718c67d91'
filter_df = df_dropped[lebron_df]
covid_filter = covid_dropped[lebron_covid]
print(filter_df.shape)
print(covid_filter.shape)
filter_df
covid_filter

In [ ]:
X = filter_df['total_sec_played']
X_covid = covid_filter['total_sec_played']
print(X.shape)
X

In [ ]:
y = filter_df['Points']
y_covid = covid_filter['Points']
print(y.shape)
y.dtype
y


In [ ]:
def  stats(a, b):
    quartilesval = a.quantile([.25, .5, .75])
    lowerq = quartilesval[.25]
    upperq = quartilesval[.75]
    iqr = upperq-lowerq
    lowerbound = lowerq - (iqr*1.5)
    upperbound = upperq + (iqr*1.5)
    upperoutliers = b.loc[a >= upperbound]
    loweroutliers = b.loc[a <= lowerbound]
    outliers=pd.concat([upperoutliers, loweroutliers])
    print(f'The statistical analysis states that there are {len(outliers)} outliers')
    print(f'the upper bound is {upperbound} and the lower bound is {lowerbound}')
    

def linearregress(x,y,a,b):
    (slope, intercept, rvalue, pvalue, stderr)=linregress(x,y)
    regress_values = x * slope + intercept
    line_eq = 'y=' + str(round(slope,2)) +'x+' + str(round(intercept,2))
    plt.scatter(x,y, color='blue')
    plt.plot(x,regress_values,"r-")
    print(f'The r-square is {rvalue}')
    print(line_eq)
    plt.annotate(line_eq,(a,b),fontsize=15, color='red')
    plt.grid()
    
def linearregress2(x,y,a,b):
    (slope, intercept, rvalue, pvalue, stderr)=linregress(x,y)
    regress_values = x * slope + intercept
    line_eq = 'y=' + str(round(slope,2)) +'x+' + str(round(intercept,2))
    plt.scatter(x,y, color='red')
    plt.plot(x,regress_values,"r-")
    print(f'The r-square is for post Covid is {rvalue}')
    print(line_eq)
    plt.annotate(line_eq,(a,b),fontsize=15, color='green')
    plt.grid()

In [ ]:

plt.xlabel('Seconds_Played')
plt.ylabel('Points_Scored')
plt.title('Points vs Time Played')
linearregress(X, y, 1, 1)
stats(filter_df['Points'], filter_df)
linearregress2(X_covid, y_covid, 25, 25)
stats(covid_filter['Points'], covid_filter)

In [35]:
df1 = pd.read_csv('data_files/FinalDataFiles/Top50WNBA.csv')
df1

,PLAYER,EFF,GP,MIN,PTS,FGM,FGA,FG%,3PM,3PA,...,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV
0,BREANNA STEWART,23.4,20,30.4,19.7,6.7,14.8,45.1,1.8,4.8,...,4.7,5.2,89.4,1.0,7.3,8.3,3.6,1.7,1.3,2.5
1,A'JA WILSON,23.0,22,31.7,20.5,7.5,15.7,48.0,0.0,0.0,...,5.4,6.9,78.1,1.5,7.0,8.5,2.0,1.2,2.0,1.6
2,NAPHEESA COLLIER,22.7,22,34.2,16.1,6.2,11.8,52.3,0.9,2.2,...,2.9,3.5,82.9,2.8,6.3,9.0,3.3,1.8,1.3,2.7
3,CANDACE PARKER,22.2,22,30.0,14.7,5.6,11.0,51.0,0.9,2.2,...,2.6,3.5,73.1,1.7,8.0,9.7,4.6,1.2,1.2,2.9
4,ALYSSA THOMAS,21.6,21,32.8,15.5,5.8,11.6,50.0,0.0,0.0,...,3.9,5.6,68.6,1.9,7.2,9.0,4.8,2.0,0.3,2.5
5,MYISHA HINES-ALLEN,21.0,22,30.0,17.0,6.8,13.4,51.0,1.2,2.8,...,2.2,2.6,82.8,1.6,7.3,8.9,2.6,1.5,0.2,2.2
6,COURTNEY VANDERSLOOT,20.9,22,31.5,13.6,4.9,9.9,49.1,1.4,3.5,...,2.5,2.9,88.9,0.5,3.0,3.5,10.0,1.2,0.4,2.5
7,DEWANNA BONNER,20.5,22,33.3,19.7,6.7,16.0,42.2,1.2,4.9,...,5.0,5.6,89.5,1.5,6.3,7.8,3.0,1.7,0.5,2.4
8,DIANA TAURASI,18.7,19,28.1,18.7,5.1,12.4,40.9,3.2,8.8,...,5.4,5.9,91.2,0.1,4.1,4.2,4.5,1.0,0.4,2.3
9,DEARICA HAMBY,17.5,22,28.3,13.0,5.0,9.3,53.9,0.8,1.7,...,2.2,3.0,71.6,1.3,5.9,7.1,2.7,1.7,0.2,2.0


In [36]:
df1['PLAYER'] = df1['PLAYER'].str.lower()
df1['PLAYER'] = df1['PLAYER'].str.title()
df1

,PLAYER,EFF,GP,MIN,PTS,FGM,FGA,FG%,3PM,3PA,...,FTM,FTA,FT%,OREB,DREB,REB,AST,STL,BLK,TOV
0,Breanna Stewart,23.4,20,30.4,19.7,6.7,14.8,45.1,1.8,4.8,...,4.7,5.2,89.4,1.0,7.3,8.3,3.6,1.7,1.3,2.5
1,A'Ja Wilson,23.0,22,31.7,20.5,7.5,15.7,48.0,0.0,0.0,...,5.4,6.9,78.1,1.5,7.0,8.5,2.0,1.2,2.0,1.6
2,Napheesa Collier,22.7,22,34.2,16.1,6.2,11.8,52.3,0.9,2.2,...,2.9,3.5,82.9,2.8,6.3,9.0,3.3,1.8,1.3,2.7
3,Candace Parker,22.2,22,30.0,14.7,5.6,11.0,51.0,0.9,2.2,...,2.6,3.5,73.1,1.7,8.0,9.7,4.6,1.2,1.2,2.9
4,Alyssa Thomas,21.6,21,32.8,15.5,5.8,11.6,50.0,0.0,0.0,...,3.9,5.6,68.6,1.9,7.2,9.0,4.8,2.0,0.3,2.5
5,Myisha Hines-Allen,21.0,22,30.0,17.0,6.8,13.4,51.0,1.2,2.8,...,2.2,2.6,82.8,1.6,7.3,8.9,2.6,1.5,0.2,2.2
6,Courtney Vandersloot,20.9,22,31.5,13.6,4.9,9.9,49.1,1.4,3.5,...,2.5,2.9,88.9,0.5,3.0,3.5,10.0,1.2,0.4,2.5
7,Dewanna Bonner,20.5,22,33.3,19.7,6.7,16.0,42.2,1.2,4.9,...,5.0,5.6,89.5,1.5,6.3,7.8,3.0,1.7,0.5,2.4
8,Diana Taurasi,18.7,19,28.1,18.7,5.1,12.4,40.9,3.2,8.8,...,5.4,5.9,91.2,0.1,4.1,4.2,4.5,1.0,0.4,2.3
9,Dearica Hamby,17.5,22,28.3,13.0,5.0,9.3,53.9,0.8,1.7,...,2.2,3.0,71.6,1.3,5.9,7.1,2.7,1.7,0.2,2.0


In [37]:
df1.to_csv('data_files/FinalDataFiles/Top50WNBA.csv')
